In [ ]:
#!/usr/bin/env python
# coding: utf-8

# Qiskit Gradient Framework using Primitives<br>
This tutorial demonstrates the use of the `qiskit.algorithms.gradients` module to evaluate quantum gradients using the [Qiskit Primitives](https://qiskit.org/documentation/apidoc/primitives.html).<br>
<br>
## Introduction<br>
The gradient frameworks allows the evaluation of quantum gradients (see [Schuld et al.](https://journals.aps.org/pra/abstract/10.1103/PhysRevA.99.032331) and [Mari et al.](https://journals.aps.org/pra/abstract/10.1103/PhysRevA.103.012405)). <br>
Besides gradients of expectation values of the form<br>
$$ \langle\psi\left(\theta\right)|\hat{O}\left(\omega\right)|\psi\left(\theta\right)\rangle $$<br>
and sampling probabilities of the form<br>
$$p_j(\theta) = |\langle j | \psi(\theta)\rangle|^2$$<br>
the gradient frameworks also support the evaluation of the [Quantum Geometric Tensor](https://quantum-journal.org/papers/q-2020-05-25-269/) (QGT) and [Quantum Fisher Information](https://quantum-journal.org/papers/q-2021-09-09-539/) (QFI) of quantum states $|\psi\left(\theta\right)\rangle$.

## A quick refresher on Qiskit Primitives<br>
<br>
The Qiskit Primitives work as an abstraction level between algorithms and (real/simulated) quantum devices. Instead of having to manually deal with tasks such as parameter binding or circuit transpilation, the `primitives` module offers a `Sampler` and an `Estimator` class that take the circuits, the observable Hamiltonians, and the circuit parameters and return the sampling distribution and the computed expectation values respectively.<br>
<br>
`qiskit.primitives` provides two classes for evaluating the circuit:<br>
- The `Estimator` class allows to evaluate expectation values of observables with respect to states prepared by quantum circuits.<br>
- The `Sampler` class returns quasi-probability distributions as a result of sampling quantum circuits.

## The `algorithms.gradients` Framework<br>
<br>
The `algorithms.gradients` module contains tools to compute both circuit gradients and circuit metrics. The gradients extend the `BaseEstimatorGradient` and `BaseSamplerGradient` base classes. These are abstract classes on top of which different gradient methods have been implemented. The methods currently available in this module are:<br>
- Parameter Shift Gradients<br>
- Finite Difference Gradients<br>
- Linear Combination of Unitaries Gradients<br>
- Simultaneous Perturbation Stochastic Approximation (SPSA) Gradients<br>
<br>
Additionally, the module offers reverse gradients for efficient classical computations.<br>
<br>
The metrics available are based on the notion of the Quantum Geometric Tensor (QGT). There is a `BaseQGT` class (`Estimator`-based) on top of which different QGT methods have been implemented:<br>
- Linear Comination of Unitaries QGT<br>
- Reverse QGT (classical)<br>
As well as a Quantum Fisher Information class (QFI) that is initialized with a reference QGT implementation from the above list.

![gradients_qiskit_algorithms.png](attachment:gradients_qiskit_algorithms.png)

<center>The outline of the <code>qiskit.algorithms.gradients</code> framework </center>

## Gradients<br>
<br>
<br>
Given a parameterized quantum state $|\psi\left(\theta\right)\rangle = V\left(\theta\right)|\psi\rangle$ with input state $|\psi\rangle$, , we want to compute either its expectation gradient $\langle\psi\left(\theta\right)|\hat{O}\left(\omega\right)|\psi\left(\theta\right)\rangle$ or the gradient of the sampling probability $p_j(\theta) = |\langle j | \psi(\theta)\rangle|^2$<br>
<br>
### Sampling gradients<br>
The formula for the gradient of sampling is:<br>
<br>
$$\frac{\partial}{\partial \theta} p_j(\theta) = \frac{\partial}{\partial \theta} |\langle j | \psi(\theta)\rangle|^2 $$<br>
<br>
Thus, the output of the sampler gradient is a list of dictionaries, where each dictionary has entries for different values of $j$ in the formula above:<br>
<br>
```<br>
[{d/d theta_1 p_1: .., d/d theta_1 p_2, ..,}, {d/d theta_2 p_1: .., d/d theta_2 p_2, ..}, ..]<br>
```<br>
<br>
### Expectation gradients<br>
The formula for expectation gradient is:<br>
<br>
$$\frac{\partial}{\partial \theta} \langle E \rangle = \frac{\partial}{\partial \theta} \langle\psi\left(\theta\right)|\hat{O}\left(\omega\right)|\psi\left(\theta\right)\rangle $$<br>
<br>
Thus, the output format of the estimator gradient is a list of derivatives:<br>
```<br>
[d/d theta_1 <E>, d/d theta_2 <E>, ...]<br>
```

## Gradient Evaluation of Quantum Circuits <br>
- Let's say that we want to use one of our `Estimator` gradients classes, then we need a quantum state $\vert\psi(\theta)\rangle$ and a Hamiltonian H acting as an observable. For the Sampler gradients, we just need a quantum state.

- We then construct a list of the parameters for which we aim to evaluate the gradient.

In[65]:

In [ ]:
from qiskit.circuit import QuantumCircuit, QuantumRegister, Parameter
from qiskit.quantum_info import SparsePauliOp
import numpy as np

nstantiate the quantum circuit

In [ ]:
a = Parameter('a')
b = Parameter('b')
q = QuantumRegister(1)
qc = QuantumCircuit(q)
qc.h(q)
qc.rz(a, q[0])
qc.rx(b, q[0])

In [ ]:
display(qc.draw('mpl'))

nstantiate the Hamiltonian observable 2X+Z

In [ ]:
H = SparsePauliOp.from_list([('X', 2), ('Z',1)])

arameter list

In [ ]:
params = [[np.pi / 4, 0]]

We can now choose a gradient type to evaluate the gradient of the circuit ansatz.

### Parameter Shift Gradients<br>
<br>
#### Using Estimator<br>
Given a Hermitian operator $g$ with two unique eigenvalues $\pm r$ which acts as generator for a parameterized quantum gate $$G(\theta)= e^{-i\theta g}.$$<br>
Then, quantum gradients can be computed by using eigenvalue $r$ dependent shifts to parameters. <br>
All [standard, parameterized Qiskit gates](https://github.com/Qiskit/qiskit-terra/tree/master/qiskit/circuit/library/standard_gates) can be shifted with $\pi/2$, i.e.,<br>
 $$ \frac{\partial\langle\psi\left(\theta\right)|\hat{O}\left(\omega\right)|\psi\left(\theta\right)\rangle}{\partial\theta} =  <br>
 \frac{\langle\psi\left(\theta+\pi/2\right)|\hat{O}\left(\omega\right)|\psi\left(\theta+\pi/2\right)\rangle -\langle\psi\left(\theta-\pi/2\right)|\hat{O}\left(\omega\right)|\psi\left(\theta-\pi/2\right)\rangle }{ 2}.$$

In[73]:

In [ ]:
from qiskit.primitives import Estimator
from qiskit.algorithms.gradients import ParamShiftEstimatorGradient

efine the estimator

In [ ]:
estimator = Estimator()
#Define the gradient
gradient = ParamShiftEstimatorGradient(estimator)

Evaluate the gradient of the circuits using parameter shift gradients

In [ ]:
pse_grad_result = gradient.run(qc, H,  params).result().gradients

In [ ]:
print('State estimator gradient computed with parameter shift', pse_grad_result)

#### Using Sampler<br>
<br>
Following a similar logic to the estimator gradient, when we have a quantum state prepared by a quantum circuit, we can shift the parametrized gates by $\pm \pi/2$ and sample to compute the gradient of the sampling probability.

In[62]:

Instantiate the quantum state with two parameters

In [ ]:
a = Parameter('a')
b = Parameter('b')

In [ ]:
q = QuantumRegister(1)
qc_sample = QuantumCircuit(q)
qc_sample.h(q)
qc_sample.rz(a, q[0])
qc_sample.rx(b, q[0])
qc_sample.measure_all() #important for sampler

In [ ]:
qc_sample.draw('mpl')

In[60]:

In [ ]:
from qiskit.primitives import Sampler
from qiskit.algorithms.gradients import ParamShiftSamplerGradient

In [ ]:
param_vals = [[np.pi/4, np.pi/2]]
sampler = Sampler()
gradient = ParamShiftSamplerGradient(sampler)
pss_grad_result = gradient.run(qc_sample, param_vals).result().gradients
print('State sampler gradient computed with parameter shift', pss_grad_result)

> **Note:** All the following methods in this tutorial are explained using the `Estimator` class to evaluate the gradients, but, in an analogous way to the Parameter Shift gradients just introduced, the method explanation can also be applied to `Sampler`-based gradients. Both versions are available in `algorithms.gradients`.

### Linear Combination of Unitaries Gradients<br>
Unitaries can be written as $U\left(\omega\right) = e^{iM\left(\omega\right)}$, where $M\left(\omega\right)$ denotes a parameterized Hermitian matrix. <br>
Further, Hermitian matrices can be decomposed into weighted sums of Pauli terms, i.e., $M\left(\omega\right) = \sum_pm_p\left(\omega\right)h_p$ with $m_p\left(\omega\right)\in\mathbb{R}$ and $h_p=\bigotimes\limits_{j=0}^{n-1}\sigma_{j, p}$ for $\sigma_{j, p}\in\left\{I, X, Y, Z\right\}$ acting on the $j^{\text{th}}$ qubit. Thus, the gradients of <br>
$U_k\left(\omega_k\right)$ are given by<br>
<br>
$$\frac{\partial U_k\left(\omega_k\right)}{\partial\omega_k} = \sum\limits_pi \frac{\partial m_{k,p}\left(\omega_k\right)}{\partial\omega_k}U_k\left(\omega_k\right)h_{k_p}$$<br>
<br>
Combining this observation with a circuit structure presented in [Simulating physical phenomena by quantum networks](https://journals.aps.org/pra/abstract/10.1103/PhysRevA.65.042323) allows us to compute the gradient with the evaluation of a single quantum circuit.

In[74]:

In [ ]:
from qiskit.algorithms.gradients import LinCombEstimatorGradient

Evaluate the gradient of the circuits using linear combination of unitaries

In [ ]:
state_grad = LinCombEstimatorGradient(estimator)

Evaluate the gradient

In [ ]:
lce_grad_result = state_grad.run(qc, H, params).result().gradients
print('State estimator gradient computed with the linear combination method', lce_grad_result)

### Finite Difference Gradients<br>
<br>
Unlike the other methods, finite difference gradients are numerical estimations rather than analytical values.<br>
This implementation employs a central difference approach with $\epsilon \ll 1$<br>
$$ \frac{\partial\langle\psi\left(\theta\right)|\hat{O}\left(\omega\right)|\psi\left(\theta\right)\rangle}{\partial\theta} \approx \frac{1}{2\epsilon} \left(\langle\psi\left(\theta+\epsilon\right)|\hat{O}\left(\omega\right)|\psi\left(\theta+\epsilon\right)\rangle - \partial\langle\psi\left(\theta-\epsilon\right)|\hat{O}\left(\omega\right)|\psi\left(\theta-\epsilon\right)\rangle\right).$$<br>
 Probability gradients are computed equivalently.

In[75]:

In [ ]:
from qiskit.algorithms.gradients import FiniteDiffEstimatorGradient

In [ ]:
state_grad = FiniteDiffEstimatorGradient(estimator, epsilon = 0.001)

Evaluate the gradient

In [ ]:
fde_grad_result = state_grad.run(qc, H, params).result().gradients
print('State estimator gradient computed with finite difference', fde_grad_result)

### SPSA Gradients<br>
<br>
SPSA gradients compute the gradients of the expectation value by the [Simultaneous Perturbation Stochastic Approximation (SPSA) algorithm](https://ieeexplore.ieee.org/document/880982). `epsilon` is the amount of offset, `batch_size` is the number of times the circuit is executed to estimate the gradient. As SPSA is a random process, use the `seed` value to avoid randomization.

In[70]:

In [ ]:
from qiskit.algorithms.gradients import SPSAEstimatorGradient

In [ ]:
state_grad = SPSAEstimatorGradient(estimator, epsilon = 0.001, batch_size=10, seed=50)

Evaluate the gradient

In [ ]:
spsae_grad_result = state_grad.run(qc, H, params).result().gradients
print('State estimator gradient computed with SPSA:', spsae_grad_result)

## Circuit Quantum Geometric Tensor (QGTs)<br>
[Quantum Geometric Tensor](https://arxiv.org/abs/1012.1337) is a metric in geometric quantum computing and can be regarded as a metric measuring the geodesic distance of points lying on the Bloch sphere. Its real and imaginary parts give different informations about the quantum state. <br>
<br>
The entries of the QGT for a pure state is given by<br>
<br>
$$QGT_{kl}(\theta) = \langle\partial_k\psi(\theta)|\partial_l\psi(\theta)\rangle-\langle\partial_k\psi(\theta)|\psi(\theta)\rangle\langle\psi(\theta)|\partial_l\psi(\theta)\rangle.$$ <br>
<br>
### Linear Combination QGT<br>
This method employs a linear combination of unitaries, as explained in the **Gradients** section.

In[71]:

In [ ]:
from qiskit.algorithms.gradients import DerivativeType, LinCombQGT

In [ ]:
qgt = LinCombQGT(estimator, derivative_type=DerivativeType.COMPLEX)

In [ ]:
param_vals = [[np.pi/4, 0.1]]

valuate the QGTs

In [ ]:
qgt_result = qgt.run(qc, param_vals).result().qgts
print('QGT:')
print(qgt_result)

### Quantum Fisher Information (QFI)<br>
<br>
[Quantum Fisher Information](https://quantum-journal.org/papers/q-2020-05-25-269/) is a metric tensor which is representative for the representation capacity of a <br>
parameterized quantum state $|\psi\left(\theta\right)\rangle = V\left(\theta\right)|\psi\rangle$ with input state $|\psi\rangle$, parametrized Ansatz $V\left(\theta\right)$.<br>
<br>
The QFI can thus be evaluated from QGT as<br>
<br>
$$<br>
\begin{align*}<br>
QFI_{kl} &= 4 * \text{Re}(QGT_{kl}) \\<br>
         &=4 *\text{Re}\left[\langle\partial_k\psi|\partial_l\psi\rangle-\langle\partial_k\psi|\psi\rangle\langle\psi|\partial_l\psi\rangle \right].<br>
\end{align*}$$

In[72]:

In [ ]:
from qiskit.algorithms.gradients import QFI

efine the QFI metric for the QGT

In [ ]:
qfi = QFI(qgt)

Evaluate the QFI

In [ ]:
qfi_result = qfi.run(qc, param_vals).result().qfis
print('QFI:')
print(qfi_result)

## Application Example: VQE with gradient-based optimization

### Estimator

Let's see an application of these gradient classes in a gradient-based optimization. We will use the Variational Quantum Eigensolver (VQE) algorith. First, the Hamiltonian and wavefunction ansatz are initialized.

In[61]:

In [ ]:
from qiskit.circuit import ParameterVector

Instantiate the system Hamiltonian

In [ ]:
h2_hamiltonian = SparsePauliOp.from_list([('II', -1.05), 
                                          ('IZ',  0.39), 
                                          ('ZI', -0.39), 
                                          ('ZZ', -0.01)])

This is the target energy

In [ ]:
h2_energy = -1.85727503

Define the Ansatz

In [ ]:
wavefunction = QuantumCircuit(2)
params = ParameterVector('theta', length=8)
it = iter(params)
wavefunction.ry(next(it), 0)
wavefunction.ry(next(it), 1)
wavefunction.rz(next(it), 0)
wavefunction.rz(next(it), 1)
wavefunction.cx(0, 1)
wavefunction.ry(next(it), 0)
wavefunction.ry(next(it), 1)
wavefunction.rz(next(it), 0)
wavefunction.rz(next(it), 1)

In [ ]:
wavefunction.draw('mpl')

In[48]:

ake circuit copies for different VQEs

In [ ]:
wavefunction_1 = wavefunction.copy()
wavefunction_2 = wavefunction.copy()

The `VQE` will take an `Estimator`, the ansatz and optimizer, and an optional gradient. We will use the `LinCombEstimatorGradient` gradient to compute the VQE.

In[55]:

In [ ]:
from qiskit.algorithms.optimizers import CG
from qiskit.algorithms.minimum_eigensolvers import VQE, SamplingVQE

onjugate Gradient algorithm

In [ ]:
optimizer = CG(maxiter=50)

Gradient callable

In [ ]:
estimator = Estimator()
grad = LinCombEstimatorGradient(estimator) # optional estimator gradient
vqe = VQE(estimator=estimator, ansatz=wavefunction, optimizer=optimizer, gradient=grad)

In [ ]:
result = vqe.compute_minimum_eigenvalue(h2_hamiltonian)
print('Result of Estimator VQE:', result.optimal_value, '\nReference:', h2_energy)

### Classical Optimizer

We can also use a classical optimizer to optimize the VQE. We'll use the `minimize` function from SciPy. 

In[52]:

In [ ]:
from scipy.optimize import minimize

lassical optimizer

In [ ]:
vqe_classical = VQE(estimator=estimator, ansatz=wavefunction_2, optimizer=minimize, gradient=grad)

In [ ]:
result_classical = vqe_classical.compute_minimum_eigenvalue(h2_hamiltonian)
print('Result of classical optimizer:', result_classical.optimal_value, '\nReference:', h2_energy)

In[53]:

In [ ]:
import qiskit.tools.jupyter
get_ipython().run_line_magic('qiskit_version_table', '')
get_ipython().run_line_magic('qiskit_copyright', '')